In [71]:
echo "I have a secret" > plaintext.txt

In [72]:
aws kms list-keys

{
    "Keys": [
        {
            "KeyId": "17b221af-0d0b-4810-87e0-cd932872b1fb",
            "KeyArn": "arn:aws:kms:ap-northeast-1:696544121619:key/17b221af-0d0b-4810-87e0-cd932872b1fb"
        },
        {
            "KeyId": "32a894d9-5016-46d0-b863-f653119697e2",
            "KeyArn": "arn:aws:kms:ap-northeast-1:696544121619:key/32a894d9-5016-46d0-b863-f653119697e2"
        },
        {
            "KeyId": "40887eff-f484-44e0-a45d-ba76c576c9bf",
            "KeyArn": "arn:aws:kms:ap-northeast-1:696544121619:key/40887eff-f484-44e0-a45d-ba76c576c9bf"
        },
        {
            "KeyId": "8c35541b-b7a2-4f98-b2a7-6d6289661550",
            "KeyArn": "arn:aws:kms:ap-northeast-1:696544121619:key/8c35541b-b7a2-4f98-b2a7-6d6289661550"
        },
        {
            "KeyId": "9a3430bd-36d2-45a6-aa84-dc87c368f6ec",
            "KeyArn": "arn:aws:kms:ap-northeast-1:696544121619:key/9a3430bd-36d2-45a6-aa84-dc87c368f6ec"
        },
        {
            "KeyId": "a7db32a5-7d03-45e

In [73]:
keyid=e871bd4f-4779-4aad-b69a-186414ca28f1

In [74]:
aws kms generate-data-key \
--key-id $keyid \
--key-spec AES_256

{
    "CiphertextBlob": "AQIDAHi1IgEEQK830+lL+fs39JI3mBXInPv1H5hoBIVC498W9wEWfVVhmRnVSIQmnCTqT3WKAAAAfjB8BgkqhkiG9w0BBwagbzBtAgEAMGgGCSqGSIb3DQEHATAeBglghkgBZQMEAS4wEQQMQgMIOohpoubnlukaAgEQgDsnx3rPIOYqVOmvzkP9PH0RGANY3SIUImBA7p4bhvJhgCBqbPG2xCqMLSUGPZDkNe5uI0xFakdtTo2ZVA==",
    "Plaintext": "ApQXyEJXbEx6VIf8JU+9e/DHEGdSQHmImDm851VDq/I=",
    "KeyId": "arn:aws:kms:ap-northeast-1:696544121619:key/e871bd4f-4779-4aad-b69a-186414ca28f1"
}


In [75]:
keys=($(aws kms generate-data-key \
--key-id $keyid \
--key-spec AES_256 \
--query [Plaintext,CiphertextBlob] --output text))

In [76]:
echo ${keys[0]}

RKbk9g9nM6eng+KTqFa+949Ky/KX7V+0XBZEqVgI1Ww=


In [77]:
echo ${keys[1]}

AQIDAHi1IgEEQK830+lL+fs39JI3mBXInPv1H5hoBIVC498W9wGSR+Z7T4QtEgmNTIStzyHDAAAAfjB8BgkqhkiG9w0BBwagbzBtAgEAMGgGCSqGSIb3DQEHATAeBglghkgBZQMEAS4wEQQMtQYvN3f92HeYOw9vAgEQgDuUvtWfHMUxW5bu7tJ+pL3Yv6/GfF0yiEdLTRhTenaHQhLcv4wcgQ+t/TIs4QOwc1y6q6Is9ALKjGUJKQ==


In [78]:
key=$(echo -n ${keys[0]} | base64 --decode \
| hexdump -v -e '/1 "%02X"')

In [79]:
echo $key

44A6E4F60F6733A7A783E293A856BEF78F4ACBF297ED5FB45C1644A95808D56C


In [80]:
aws kms generate-random --number-of-bytes 16

{
    "Plaintext": "c7vEm0bfFaceje5w1Ps4+A=="
}


In [81]:
iv=$(aws kms generate-random \
--number-of-bytes 16 \
--query Plaintext --output text | \
base64 --decode | \
hexdump -v -e '/1 "%02X"')

In [82]:
echo $iv

BFC3A2E63F8E8A6D65C40755B123BBDB


In [83]:
echo "Encrypted data key: ${keys[1]}" > encrypted.txt

In [84]:
echo "Initialization vector: ${iv}" >> encrypted.txt

In [85]:
openssl aes-256-cbc -iv $iv -K $key \
-in plaintext.txt -base64 >> encrypted.txt

In [86]:
cat encrypted.txt

Encrypted data key: AQIDAHi1IgEEQK830+lL+fs39JI3mBXInPv1H5hoBIVC498W9wGSR+Z7T4QtEgmNTIStzyHDAAAAfjB8BgkqhkiG9w0BBwagbzBtAgEAMGgGCSqGSIb3DQEHATAeBglghkgBZQMEAS4wEQQMtQYvN3f92HeYOw9vAgEQgDuUvtWfHMUxW5bu7tJ+pL3Yv6/GfF0yiEdLTRhTenaHQhLcv4wcgQ+t/TIs4QOwc1y6q6Is9ALKjGUJKQ==
Initialization vector: BFC3A2E63F8E8A6D65C40755B123BBDB
BNNku0EO/Z//NB3YHuS9E64s6X+CbjqgYJADw0MHKfc=


In [87]:
cd message

In [88]:
aws kms decrypt \
--ciphertext-blob fileb://data_key_encrypted.bin

{
    "KeyId": "arn:aws:kms:ap-northeast-1:696544121619:key/e871bd4f-4779-4aad-b69a-186414ca28f1",
    "Plaintext": "tYl4va0/5OC1cFTjWPN3WPfmhf9WbKWP8XlQi66BlTQ="
}


In [89]:
key=$(aws kms decrypt \
--ciphertext-blob fileb://data_key_encrypted.bin \
--query Plaintext --output text | \
base64 --decode | \
hexdump -v -e '/1 "%02X"')

In [90]:
echo $key

B58978BDAD3FE4E0B57054E358F37758F7E685FF566CA58FF179508BAE819534


In [91]:
iv=$(cat iv_hex.txt)

In [92]:
openssl aes-256-cbc -d \
-in message_encrypted_base64.txt \
-iv $iv -K $key -base64

 ______________
< Hello World! >
 --------------
   \
    \
        .--.
       |o_o |
       |:_/ |
      //   \ \
     (|     | )
    /'\_   _/`\
    \___)=(___/



In [93]:
unset iv
unset key

# オプションの課題 1

In [94]:
cd ..

In [95]:
cat encrypted.txt

Encrypted data key: AQIDAHi1IgEEQK830+lL+fs39JI3mBXInPv1H5hoBIVC498W9wGSR+Z7T4QtEgmNTIStzyHDAAAAfjB8BgkqhkiG9w0BBwagbzBtAgEAMGgGCSqGSIb3DQEHATAeBglghkgBZQMEAS4wEQQMtQYvN3f92HeYOw9vAgEQgDuUvtWfHMUxW5bu7tJ+pL3Yv6/GfF0yiEdLTRhTenaHQhLcv4wcgQ+t/TIs4QOwc1y6q6Is9ALKjGUJKQ==
Initialization vector: BFC3A2E63F8E8A6D65C40755B123BBDB
BNNku0EO/Z//NB3YHuS9E64s6X+CbjqgYJADw0MHKfc=


In [96]:
# encrypted.txt の内容を変数に代入する
encrypted_key='AQIDAHi1IgEEQK830+lL+fs39JI3mBXInPv1H5hoBIVC498W9wHlrc4re/lwMVIVYWUspLziAAAAfjB8BgkqhkiG9w0BBwagbzBtAgEAMGgGCSqGSIb3DQEHATAeBglghkgBZQMEAS4wEQQMZSIa+XpaufeFgaq4AgEQgDtjoHMR+BKaf84hWHSh5ifXR5jzPiQcPUA7DkoZB4HY2AVvdIx8rcFaGebCwz9k9B2Xuwxv+NKbCKBw0w=='
iv='AEB03F5F6884558A604A948D9DA627A1'
encrypted_text='4N9IfotdCn/qVSjfmTaA0lK1/xwkoOnG6H6aT8Qd2ms='

In [97]:
# 暗号化キーをbase64デコードする(バイナリに戻す)
echo $encrypted_key | base64 --decode > key.bin

In [98]:
# 生成された暗号化キーファイルを確認
ls -l key.bin

-rw-r--r-- 1 root root 184 Nov 13 07:24 key.bin


In [99]:
# KMSで暗号化キーを復号する
aws kms decrypt --ciphertext-blob fileb://key.bin

{
    "KeyId": "arn:aws:kms:ap-northeast-1:696544121619:key/e871bd4f-4779-4aad-b69a-186414ca28f1",
    "Plaintext": "+FjsTFqSGbHLmgV9LP7aNg4SQuN21VYwZN/UsmUOkZo="
}


In [103]:
# 暗号化キーを復号し、OpenSSLで利用できる形式に変換し、key 変数に入れる
key=$(aws kms decrypt --ciphertext-blob fileb://key.bin \
--query 'Plaintext' --output text \
| base64 --decode |hexdump -v -e '/1 "%02X"')

In [104]:
# 復号されたキーを確認
echo $key

F858EC4C5A9219B1CB9A057D2CFEDA360E1242E376D5563064DFD4B2650E919A


In [105]:
# OpenSSLで復号を行う
echo $encrypted | openssl aes-256-cbc -d \
-iv $iv -K $key -base64

I have a secret
